In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
DATA_FOLDER = '../data/'
DATA_ZIP = DATA_FOLDER + 'datasets.zip'

DATA_TRAIN_PATH = DATA_FOLDER + 'train.csv'
DATA_TRAIN_PATH_CLEAN = DATA_FOLDER + 'train_clean.csv'
DATA_TEST_PATH = DATA_FOLDER + 'test.csv' 

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
from split_data import *

y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
y_clean, tX_clean, ids_clean = load_csv_data(DATA_TRAIN_PATH_CLEAN)

split_ratio = 0.8
tX_train, tX_validation, y_train, y_validation = split_data(tX, y, split_ratio)
tX_train_clean, tX_validation_clean, y_train_clean, y_validation_clean = split_data(tX_clean, y_clean, split_ratio)

In [4]:
print("Raw set : ")
row_format = "{:<15}" * 3
print(row_format.format("", "Training", "Validation"))
print(row_format.format("Features", str(tX_train.shape), str(tX_validation.shape)))
print(row_format.format("Labels", str(y_train.shape), str(y_validation.shape)))

print("\nClean set : ")
row_format = "{:<15}" * 3
print(row_format.format("", "Training", "Validation"))
print(row_format.format("Features", str(tX_train_clean.shape), str(tX_validation_clean.shape)))
print(row_format.format("Labels", str(y_train_clean.shape), str(y_validation_clean.shape)))

Raw set : 
               Training       Validation     
Features       (200000, 30)   (50000, 30)    
Labels         (200000,)      (50000,)       

Clean set : 
               Training       Validation     
Features       (60122, 23)    (15031, 23)    
Labels         (60122,)       (15031,)       


## Do your thing crazy machine learning thing here :) ...

In [20]:
from stochastic_gradient_descent import *
from gradient_descent import *
from costs import *
# weights_clean, t_loss_clean = least_squares(y_train_clean, tX_train_clean)
# weights, t_loss = least_squares(y_train, tX_train)
initial_w = np.zeros(tX_train_clean.shape[1])
max_iters = 50
gamma = 0.01
batch_size=10
losses, weights = gradient_descent(y_train_clean, tX_train_clean, initial_w, max_iters, gamma)
weight_clean = weights[-1]

Gradient Descent(0/49): loss=0.5, w0=0.0038923184473918355, w1=-0.002478739706425339
Gradient Descent(1/49): loss=0.4934845902115996, w0=0.007680747564935643, w1=-0.004905564156974364
Gradient Descent(2/49): loss=0.48734514202299334, w0=0.011368188356217699, w1=-0.0072818876359016195
Gradient Descent(3/49): loss=0.481557932855642, w0=0.014957457335966502, w1=-0.00960907846090936
Gradient Descent(4/49): loss=0.4761008566424111, w0=0.01845128911508111, w1=-0.011888460662830693
Gradient Descent(5/49): loss=0.47095330629775894, w0=0.021852338901950794, w1=-0.014121315599300181
Gradient Descent(6/49): loss=0.46609606517732843, w0=0.025163184922946988, w1=-0.016308883505120567
Gradient Descent(7/49): loss=0.46151120681354457, w0=0.02838633076486188, w1=-0.018452364981920304
Gradient Descent(8/49): loss=0.4571820022718942, w0=0.03152420764196883, w1=-0.020552922429587568
Gradient Descent(9/49): loss=0.45309283452581534, w0=0.0345791765902808, w1=-0.022611681421861877
Gradient Descent(10/49): 

In [21]:
# print("Training loss : {}".format(compute_loss(y_train, tX_train, weights)))
# print("Validation loss : {}".format(compute_loss(y_validation, tX_validation, weights)))
print("Clean training loss : {}".format(compute_loss(y_train_clean, tX_train_clean, weight_clean)))
print("Clean validation loss : {}".format(compute_loss(y_validation_clean, tX_validation_clean, weight_clean)))

Clean training loss : 0.38695257291710555
Clean validation loss : 0.38841011798491126


## Generate predictions and save ouput in csv format for submission:

In [ ]:
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = DATA_FOLDER + 'submission.csv' 
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)